In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Big Query

In [2]:
from google.cloud import bigquery

In [3]:
client=bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [6]:
tables = list(client.list_tables(dataset))

In [7]:
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


In [8]:
table_ref = dataset_ref.table("full")
table = client.get_table(table_ref)

In [10]:
table.description

'A full daily update of all the stories and comments in Hacker News.'

In [11]:
table.num_rows

27708712

## Table Schema

In [12]:
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), None),
 SchemaField

In [13]:
client.list_rows(table,max_results=5)

In [14]:
client.list_rows(table,max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,&gt;I&#x27;m done with this conversation since...,None,noobly,NaN,1541023247,2018-10-31 22:00:47+00:00,comment,18349611,18348253,NaN,NaN,None
1,None,None,What do you mean by sync being hard to impleme...,None,e_proxus,NaN,1434378064,2015-06-15 14:21:04+00:00,comment,9719409,9719293,NaN,NaN,None
2,None,None,"This is pretty dumb and naive, and loaded with...",None,Retra,NaN,1434378063,2015-06-15 14:21:03+00:00,comment,9719408,9719205,NaN,NaN,None
3,None,None,5. Biker not using lights at night &quot;becau...,None,rjsw,NaN,1434378022,2015-06-15 14:20:22+00:00,comment,9719401,9719372,NaN,NaN,None
4,None,None,But they are all either very inexpensive or th...,None,g8gggu89,NaN,1434378020,2015-06-15 14:20:20+00:00,comment,9719400,9719006,NaN,NaN,None


In [15]:
client.list_rows(table, selected_fields=table.schema[:1], max_results = 5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,title
0,None
1,None
2,None
3,None
4,None
